In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy import signal

### Read this First

#### Remember that `tab` is is useful for autocompletion.

#### Remember that `shift + tab` is useful for rapidly obtaining usage + documentation.

###**3D Camera Geometry**

**In class we covered material for single and multi-camera systems. This notebook will help ground some of these geometric concepts.**

**First, we'll define some of the components helpful for for camera geometry**

In [ ]:
def rotatex(t) :
  """ Creates a 3D rotation matrix representing a rotation about the x-axis by t radians """
  return np.array([[1,0,0],[0,np.cos(t),-np.sin(t)],[0,np.sin(t),np.cos(t)]])

def rotatey(t) :
  """ Creates a 3D rotation matrix representing a rotation about the y-axis by t radians """
  return np.array([[np.cos(t),0,np.sin(t)],[0,1,0],[-np.sin(t),0,np.cos(t)]])

def rotatez(t) :
  """ Creates a 3D rotation matrix representing a rotation about the z-axis by t radians """
  return [[np.cos(t),-np.sin(t),0],[np.sin(t),np.cos(t),0],[0,0,1]]

def rotate2D(t) :
  """ Creates a 2D rotation matrix representing a rotation by t radians """
  return [[np.cos(t),-np.sin(t)],[np.sin(t),np.cos(t)]]

def trans(x,y,z) :
  """ Creates a 3D translation vector of shape 3x1 """
  return np.array([[x,y,z]]).T

def pointh(x,y,z) :
  """ Creates a 3D point in homogeneous coordinates """
  return np.array([[x,y,z,1]].T)

def htrans(R,t) :
  """ Creates a 3D homogeneous transformation matrix """
  return np.concatenate((np.concatenate((R,t),axis=1),np.array([[0,0,0,1]])))

**Now, we'll define some components for camera internal geometry**

In [ ]:
def Kmatrix(sx,sy,ox,oy) :
  """ Creates the matrix K that represents the calibration matrix """
  return np.array([[sx,0,ox],[0,sy,oy],[0,0,1]])

def Mmatrix(K) :
  """ Creates the matrix M that represents the intrinsic matrix """
  return np.concatenate((K, np.array([[0,0,0]]).T),axis=1)

**Now, write a function that, given a intrinsic matrix M and a extrinsic matrix H, computes the projection matrix P.** 

In [ ]:
def Pmatrix(M, H):
  """ Creates a 3x4 projection matrix P that relates world coordinates to pixel coordinates
      Args:
        M: 3x4 matrix representing the intrinsic parameters of the camera
        H: 4x4 matrix representing the extrinsic parameters of the camera
      Return:
        P: 3x4 matrix relating world coordinates to pixel coordinates
  """
  

  return P

**Now, let's make some real world points to project.  Since this is a camera, we'll make the z axis larger than x and y**

In [ ]:
# Make 20 random 3D points stacked horizontally 
pts = np.concatenate((np.random.random([3,20]),np.ones([1,20])),axis=0)
pts[0:2,:] = (pts[0:2,:] - 0.5) * 100
pts[2,:] = (pts[2,:] - 0.5) * 100 + 1000

# Plot these points
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.scatter(pts[0],pts[1],zs=pts[2],s=5)

**Create an K matrix with parameters $s_x=s_y=1000$, $o_x=52.2$, $o_y=55.3$. Create a H matrix with identity rotation and a translation $(x, y, z)=(42, 33, 1000)$. Then, using those, create the P matrix and print the resulting matrix.**

**Transform our real world points pts using this projection matrix and store the resulting pixel coordinates into pix_pts. Remember that the results are in homogeneous coordinates so when we plot it we have to ensure that $w=1$ for hoomogeneous coordinate $(u, v, w)$**

In [ ]:
pix_pts = np.zeros((3, 20))


# Plot these points
fig = plt.figure()
plt.scatter(pix_pts[0], pix_pts[1], s=5)
plt.gca().invert_yaxis()

**Why did I invert the y axis here?**

**Now we want to compare this with the plot of pts above to see how similar our camera view with the real world. Plot pts as before using scatter, but now adjust viewpoint of the plot to have elevation as $-90$ and azimuth as $-90$.**

In [ ]:
# Plot pts again but with a different viewpoint


**Which directions are the x, y, and z axes? (i.e. which is the vertical/horizontal/in or out of plane axis? Which direction is the positive direction for each?)**

**Do your camera points look the same as the real world points? (They should)**

**Now, change the parameters such that $o_x=0$, recompute the projection matrix, reproject the real world points to the pixel coordinates and plot it as before.**

**What changed between this and the previous plot you made? What does the parameter $o_x$ represent?**

**Now change $s_y=400$ and do the same thing. Keep $o_x=0$ so you can compare without scrolling too far.**

**What changed between this and the previous plot you made? What does the parameter $s_y$ represent?**

**Now let's try and recover the projection matrix. We've already covered this in the Week 6 Python Notebook, so it should be relatively straightforward. Using the real world coordinates pts and the pixel coordinates obtained when we set $s_y=400$ and $o_x=0$ above, create the $A$ matrix in $Ap=0$ that was covered in class to solve for the projection matrix.**

In [ ]:
A = np.zeros((2 * 20, 12))


**Compute the projection matrix P by solving for $Ap=0$, and print it. Compare it with the projection matrix computed above to confirm that they are identical.**

**Recover the K and R matrices, and then the translation vector t using RQ factorization, and print them to make sure they are consistent with the parameters that you provided to create the projection matrix.**

**Now pick your own parameters for the intrinsic and extrinsic matrices, project pts, and then recover the projection matrix P. Double check that you're able to recover the matrix.**

###**Uncalibrated stereo**

**Now that we're more confident with single camera geometry, let's try relate the geometries and the viewpoints of two cameras. The goal of this is to be able to find a relationship between the location of an object in one camera view and the location of the object in the other camera view.**

**Here, let's create 20 random points again** 

In [ ]:
# Make 20 random 3D points stacked horizontally 
npoints = 20
pts = np.concatenate((np.random.random([3, npoints]),np.ones([1, npoints])),axis=0)
pts[0:2,:] = (pts[0:2,:] - 0.5) * 100
pts[2,:] = (pts[2,:] - 0.5) * 100 + 1000
colors = plt.cm.rainbow(np.linspace(0, 1, npoints))

**We also plot these points, is a couple of different views that will make more sense that the default view.**

In [ ]:
# Plot these points
fig = plt.figure(figsize=(48, 8))
ax = fig.add_subplot(131, projection='3d')
plt.scatter(pts[0],pts[1],zs=pts[2],s=21, c=colors)
ax.view_init(elev=-70, azim=-90)

ax2 = fig.add_subplot(132, projection='3d')
plt.scatter(pts[0],pts[1],zs=pts[2],s=21, c=colors)
ax2.view_init(elev=-30, azim=-90)

**Now we want to simulate two cameras. For camera 1, create a projection matrix with parameters $s_x=s_y=1000$, $o_x=35$, $o_y=55.3$, with a y-axis rotation by -0.3 radians, and a translation vector of $[280, 33, 1000]$.** 

**Project the real world pts above into the view of this camera, and plot these pixel coordinates with the same colours as above corresponding to each point (set c=colors) and remember to invert the y-axis.**

**Now simulate camera 2, by creating a projection matrix with parameters $s_x=s_y=1000$, $o_x=35$, $o_y=55.3$, with a y-axis rotation by 0.3 radians, and a translation vector of $[390, 33, 1000]$.** 

**Project the real world pts above into the view of this camera, and plot these pixel coordinates with the same colours as above corresponding to each point (set c=colors) and remember to invert the y-axis.**

**Which camera is the camera on the left? How can you tell?**

**Now we have the pixel coordiantes in both images and we know which pairs of points correspond to each other, we can solve for the fundamental matrix, which relates pixel coordinates from one image to another.**

**Below, fill in the matrix A that you find in slide 46**

In [ ]:
A = np.zeros((npoints, 9))


**Solve for the fundamental matrix given that $Af=0$, and print out the fundamental matrix F. For reference please see slide 62.**

**Try and recover the essential matrix. We can do this because you already know the calibration matrices $K_r$ and $K_l$. Display the result**

**Given the essential matrix, we can also recover the rotation between the two cameras. Compute $R$, the rotation between the two cameras, using the essential matrix $E$. Display it, and then using rotatey() display the actual rotation between the two cameras (check the extrinsic matrices of the cameras to determine the true rotation between them).**

**Are they the same? They should be.**

**Now try and recover the translation, and display the result.**

**Chances are, this will be different from the true translation between the cameras. Why is this so?**

**Now let's try and plot the epipolar lines. Write a function that given the fundamental matrix and an image pixel coordinate in either the left or the right image, returns the coefficients of the epipolar line in the other image.**

In [ ]:
def findEpipolar(F, pix_l=None, pix_r=None):
  """ Computes the epipolar line coefficients given the fundamental matrix F and either
      the pixel coordinates in the left image (pix_l) or the pixel coordinates in the 
      right image (pix_r)
      Args:
        F: Fundamenal matrix 3x3 that relates pixels in one image to another
        pix_l: Pixel coordinate in the left image (shape [3, 1])
        pix_r: Pixel coordinate in the right image(shape [1, 3])
      Return:
        line_coeff: The coefficients of the epipolar line in the other image
                    If the equation is au + bv + c = 0, the line_coeff = [a, b, c]
  """

  # If the right pixel coordinate is provided, we want to find the coefficients of
  # the line in the left image
  if pix_l is None:


  # If the left pixel coordinate is provided, we want to find the coefficients of 
  # the line in the right image
  elif pix_r is None:


  else:
    line_coeff = None

  return line_coeff

**Now complete the function below, that given the coefficient of the line and either the $x$ or the $y$ coordinate, finds the other one.**

In [ ]:
def findOther(line_coeff, u=None, v=None):
  """ Given the coefficients of au + bv + c = 0 in the form [a, b, c], compute either
      u if v is given, or v if u is given.
      Args:
        line_coeff: [a, b, c] vector of the coefficients of the equation above
        u: The x coordinate of the pixel
        v: The y coordinate of the pixel
      Return:
        The other pixel coordinate, None if nothing else is provided
  """

  if u is None:


  elif v is None:


  return None

**Now, using the two functions above, plot the epipolar lines corresponding to the points from the right image onto the left image. Plot the points in the left image in the same plot. Make sure that the colours match, and each point in the left image should have an epipolar line that passes through them.**

**Now do the same thing, plotting the epipolar lines corresponding to left image points onto the right image.**

**Now, change the extrinsics of the right camera to include another rotation in either x or z in addition to the y rotation, find the geometry between the left camera and this new right camera, and plot the epipolar lines.**